In [33]:
import numpy as np
import pandas as pd
from arch.unitroot import ADF, PhillipsPerron, DFGLS

In [34]:
#Fungsi untuk uji stasioneritas
def stasionary_test(n_simulasi, n_daftar_sampel, daftar_alpha, trend):
    penolakanH0_adf = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))
    penolakanH0_pp = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))
    penolakanH0_dfgls = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))
    alpha_empiris_adf = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))
    alpha_empiris_pp = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))
    alpha_empiris_dfgls = np.zeros((len(n_daftar_sampel), len(daftar_alpha)))

    for i, n_sampel in enumerate(n_daftar_sampel):
        for j, alpha in enumerate(daftar_alpha):
            H0_ditolak_adf = 0
            H0_ditolak_pp = 0
            H0_ditolak_dfgls = 0
            alphaempiris_adf = 0
            alphaempiris_pp = 0
            alphaempiris_dfgls = 0
            for _ in range(n_simulasi):
                np.random.seed()
                epsilon = np.random.normal(size=n_sampel)
                data = np.cumsum(epsilon) + trend * np.arange(n_sampel)

                # ADF test
                adf = ADF(data)
                if adf.pvalue < alpha:
                    H0_ditolak_adf += 1
                    alphaempiris_adf += 1

                # Phillips-Perron test
                pp = PhillipsPerron(data)
                if pp.pvalue < alpha:
                    H0_ditolak_pp += 1
                    alphaempiris_pp += 1

                # DF-GLS test
                dfgls = DFGLS(data)
                if dfgls.pvalue < alpha:
                    H0_ditolak_dfgls += 1
                    alphaempiris_dfgls += 1

            penolakanH0_adf[i, j] = H0_ditolak_adf
            penolakanH0_pp[i, j] = H0_ditolak_pp
            penolakanH0_dfgls[i, j] = H0_ditolak_dfgls
            alpha_empiris_adf[i, j] = alphaempiris_adf / n_simulasi
            alpha_empiris_pp[i, j] = alphaempiris_pp / n_simulasi
            alpha_empiris_dfgls[i, j] = alphaempiris_dfgls / n_simulasi

    return penolakanH0_adf, penolakanH0_pp, penolakanH0_dfgls, alpha_empiris_adf, alpha_empiris_pp, alpha_empiris_dfgls

In [35]:
# Parameter simulasi
n_simulasi = 1000
n_daftar_sampel = [50, 100, 150, 200, 500]
daftar_alpha = [0.01, 0.05, 0.10, 0.25, 0.5]
trend = 0.05

In [36]:
# Menghitung kekuatan uji
penolakanH0_adf, penolakanH0_pp, penolakanH0_dfgls, alpha_empiris_adf, alpha_empiris_pp, alpha_empiris_dfgls = stasionary_test(n_simulasi, n_daftar_sampel, daftar_alpha, trend)

In [37]:
# Menampilkan keseluruhan hasil simulasi
print("H0 ditolak untuk ADF test:")
print(pd.DataFrame(penolakanH0_adf, index=n_daftar_sampel, columns=daftar_alpha))
print("\nAlpha Empirisi dari ADF test:")
print(pd.DataFrame(alpha_empiris_adf, index=n_daftar_sampel, columns=daftar_alpha))

print("\nH0 ditolak untuk Phillips-Perron test:")
print(pd.DataFrame(penolakanH0_pp, index=n_daftar_sampel, columns=daftar_alpha))
print("\nAlpha Empirisi dari Phillips-Perron test:")
print(pd.DataFrame(alpha_empiris_pp, index=n_daftar_sampel, columns=daftar_alpha))

print("\nH0 ditolak untuk DF-GLS test:")
print(pd.DataFrame(penolakanH0_dfgls, index=n_daftar_sampel, columns=daftar_alpha))
print("\nAlpha Empirisi dari DF-GLS test:")
print(pd.DataFrame(alpha_empiris_dfgls, index=n_daftar_sampel, columns=daftar_alpha))

H0 ditolak untuk ADF test:
     0.01  0.05   0.10   0.25   0.50
50   29.0  87.0  130.0  279.0  507.0
100  18.0  85.0   95.0  275.0  482.0
150  14.0  46.0   92.0  240.0  484.0
200  12.0  38.0  111.0  228.0  443.0
500   5.0  42.0   79.0  209.0  403.0

Alpha Empirisi dari ADF test:
      0.01   0.05   0.10   0.25   0.50
50   0.029  0.087  0.130  0.279  0.507
100  0.018  0.085  0.095  0.275  0.482
150  0.014  0.046  0.092  0.240  0.484
200  0.012  0.038  0.111  0.228  0.443
500  0.005  0.042  0.079  0.209  0.403

H0 ditolak untuk Phillips-Perron test:
     0.01  0.05   0.10   0.25   0.50
50   18.0  60.0   78.0  247.0  505.0
100  12.0  70.0   97.0  270.0  507.0
150   9.0  43.0  104.0  254.0  487.0
200  12.0  47.0  109.0  246.0  454.0
500   8.0  44.0   90.0  224.0  409.0

Alpha Empirisi dari Phillips-Perron test:
      0.01   0.05   0.10   0.25   0.50
50   0.018  0.060  0.078  0.247  0.505
100  0.012  0.070  0.097  0.270  0.507
150  0.009  0.043  0.104  0.254  0.487
200  0.012  0.047  0.109 